In [1]:
import os
import pandas as pd
import pandasql as ps
from collections import Counter
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
path = os.getenv('path')
df = pd.read_csv(path + 'ijambo-data.csv', parse_dates=[10,11,12,13,14,15])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5378 entries, 0 to 5377
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   country               5378 non-null   object             
 1   guesses               5378 non-null   object             
 2   id                    5378 non-null   int64              
 3   score                 5378 non-null   float64            
 4   solution              5378 non-null   object             
 5   time_taken            5378 non-null   int64              
 6   timezone              5327 non-null   object             
 7   won                   5378 non-null   bool               
 8   start_time            5378 non-null   object             
 9   end_time              5378 non-null   object             
 10  list_guesses          5378 non-null   object             
 11  count_guesses         5378 non-null   object             
 12  conver

In [3]:
df.head(2)

,country,guesses,id,score,solution,time_taken,timezone,won,start_time,end_time,list_guesses,count_guesses,converted_start_time,start_date,start_hour,converted_end_time,end_date,end_hour
0,Burundi,"['ABANA', 'BWIZA', 'IKIBI', 'IROBE', 'IMIBU', ...",5378,0.0,IDUBU,35826,Africa/Bujumbura,False,2022-04-06 11:19:53 +0200,2022-04-06 21:16:59 +0200,"ABANA,BWIZA,IKIBI,IROBE,IMIBU,IYUBU",6,2022-04-06 09:19:53+00:00,2022-04-06,2022-04-06 09:19:53,2022-04-06 19:16:59+00:00,2022-04-06,19:16:59
1,Poland,"['GUTYA', 'SHORA', 'IMBWA', 'INDWI', 'INKWI']",5377,54810.0,INKWI,10962,Europe/Warsaw,True,2022-04-06 18:10:46 +0200,2022-04-06 21:13:28 +0200,"GUTYA,SHORA,IMBWA,INDWI,INKWI",5,2022-04-06 16:10:46+00:00,2022-04-06,2022-04-06 16:10:46,2022-04-06 19:13:28+00:00,2022-04-06,19:13:28


In [4]:
# players per day
q1 = """SELECT start_date, c AS count_players,

SUM(c) OVER(ORDER BY start_date) AS running_total_players
FROM (
  SELECT start_date, COUNT(DISTINCT id) c
  FROM (
    SELECT MIN(start_date) start_date, id
    FROM df
    GROUP BY id
  ) 
  GROUP BY start_date
) 
"""

print(ps.sqldf(q1, locals()))

                    start_date  count_players  running_total_players
0   1970-01-01 00:00:00.000000             46                     46
1   2022-03-28 00:00:00.000000              2                     48
2   2022-03-29 00:00:00.000000              2                     50
3   2022-03-30 00:00:00.000000             82                    132
4   2022-03-31 00:00:00.000000            723                    855
5   2022-04-01 00:00:00.000000            653                   1508
6   2022-04-02 00:00:00.000000            715                   2223
7   2022-04-03 00:00:00.000000            732                   2955
8   2022-04-04 00:00:00.000000            802                   3757
9   2022-04-05 00:00:00.000000            852                   4609
10  2022-04-06 00:00:00.000000            769                   5378


In [5]:
# players per country
q2 = """ SELECT
    DISTINCT country,
    COUNT (id) AS count_players
FROM
    df
GROUP BY
    country

ORDER BY COUNT (id) DESC

LIMIT 10  
"""

print(ps.sqldf(q2, locals()))

          country  count_players
0         Burundi           2700
1          Canada            724
2          France            664
3          Rwanda            324
4   United States            226
5           Kenya            143
6         Belgium             75
7  United Kingdom             72
8          Sweden             48
9     South Korea             46


In [21]:
# players by country per day
q3 = """ SELECT
    DISTINCT country,
    start_date,
    COUNT (DISTINCT id) AS count_players
FROM df
WHERE start_date 
GROUP BY
    country, start_date

ORDER BY start_date DESC, COUNT (DISTINCT id) DESC

LIMIT 10  
"""

print(ps.sqldf(q3, locals()))

          country                  start_date  count_players
0         Burundi  2022-04-06 00:00:00.000000            394
1          Canada  2022-04-06 00:00:00.000000             97
2          France  2022-04-06 00:00:00.000000             91
3          Rwanda  2022-04-06 00:00:00.000000             47
4   United States  2022-04-06 00:00:00.000000             40
5           Kenya  2022-04-06 00:00:00.000000             19
6         Belgium  2022-04-06 00:00:00.000000             13
7  United Kingdom  2022-04-06 00:00:00.000000             10
8          Sweden  2022-04-06 00:00:00.000000             10
9     South Korea  2022-04-06 00:00:00.000000              4


In [20]:
# rank score by country 
q4 = """ SELECT
    DISTINCT country,
    avg(score) AS avg_score 
FROM
    df
WHERE won = 1 
GROUP BY
    country
ORDER BY avg(score) DESC

LIMIT 10    
"""

print(ps.sqldf(q4, locals()))

         country     avg_score
0      Australia  4.122333e+09
1        Denmark  6.183223e+08
2        Belgium  1.555673e+08
3         Canada  9.899467e+07
4  United States  9.547158e+07
5         Rwanda  9.240824e+07
6         France  4.564601e+07
7        Burundi  2.739454e+07
8         Norway  7.265125e+04
9     Guadeloupe  5.967517e+04


In [22]:
#rank score by country per day
q5 = """SELECT
    country,
    start_date,
    avg(score)  AS avg_score
from
    (
        select *,avg(score) OVER (PARTITION BY country, start_date) 
        from df
    ) 
WHERE won = 1  
GROUP BY
     country, start_date
ORDER BY
    start_date DESC, avg(score) DESC

LIMIT 10
"""

print(ps.sqldf(q5, locals()))

       country                  start_date      avg_score
0      Senegal  2022-04-06 00:00:00.000000  179140.000000
1        Italy  2022-04-06 00:00:00.000000   49044.000000
2  South Korea  2022-04-06 00:00:00.000000   41311.333333
3      Denmark  2022-04-06 00:00:00.000000   28916.000000
4       Poland  2022-04-06 00:00:00.000000   27659.000000
5      Burundi  2022-04-06 00:00:00.000000   19983.030702
6        Egypt  2022-04-06 00:00:00.000000   18445.000000
7       France  2022-04-06 00:00:00.000000   17093.109375
8  Netherlands  2022-04-06 00:00:00.000000   16229.333333
9        China  2022-04-06 00:00:00.000000   15131.000000


In [9]:
#separate list of guesses in list_guesses column

df_expanded = df['list_guesses'].str.split(',', expand=True)
df_expanded.columns = ['guesses_sep'+str(i) for i in df_expanded.columns]

df_expanded_concat = pd.concat([df,df_expanded], axis=1)


df_reproduced = pd.melt(df_expanded_concat, id_vars=['country','guesses','id', 'score','solution', 'time_taken',
'timezone', 'won', 'start_time', 'end_time', 'list_guesses', 'count_guesses', 'converted_start_time', 'start_date',
                    'start_hour', 'converted_end_time', 'end_date', 'end_hour'],
                                   value_vars=df_expanded.columns, var_name='guesses_nber', 
                                   value_name='guesses_extracted').dropna()


df_reproduced.head()

,country,guesses,id,score,solution,time_taken,timezone,won,start_time,end_time,list_guesses,count_guesses,converted_start_time,start_date,start_hour,converted_end_time,end_date,end_hour,guesses_nber,guesses_extracted
0,Burundi,"['ABANA', 'BWIZA', 'IKIBI', 'IROBE', 'IMIBU', ...",5378,0.0,IDUBU,35826,Africa/Bujumbura,False,2022-04-06 11:19:53 +0200,2022-04-06 21:16:59 +0200,"ABANA,BWIZA,IKIBI,IROBE,IMIBU,IYUBU",6,2022-04-06 09:19:53+00:00,2022-04-06,2022-04-06 09:19:53,2022-04-06 19:16:59+00:00,2022-04-06,19:16:59,guesses_sep0,ABANA
1,Poland,"['GUTYA', 'SHORA', 'IMBWA', 'INDWI', 'INKWI']",5377,54810.0,INKWI,10962,Europe/Warsaw,True,2022-04-06 18:10:46 +0200,2022-04-06 21:13:28 +0200,"GUTYA,SHORA,IMBWA,INDWI,INKWI",5,2022-04-06 16:10:46+00:00,2022-04-06,2022-04-06 16:10:46,2022-04-06 19:13:28+00:00,2022-04-06,19:13:28,guesses_sep0,GUTYA
2,Canada,"['AMAZI', 'UMUTI', 'KURYA', 'KWICA', 'AMATA', ...",5376,0.0,IDUBU,164,America/Toronto,False,2022-04-06 15:10:27 -0400,2022-04-06 15:13:12 -0400,"AMAZI,UMUTI,KURYA,KWICA,AMATA,GUKWA",6,2022-04-06 19:10:27+00:00,2022-04-06,2022-04-06 19:10:27,2022-04-06 19:13:12+00:00,2022-04-06,19:13:12,guesses_sep0,AMAZI
3,Burundi,"['IMENA', 'IKIVI', 'ISEVU', 'IFURU', 'IRUGU', ...",5375,131904.0,IDUBU,21584,Africa/Bujumbura,True,2022-04-06 15:11:37 +0200,2022-04-06 21:11:22 +0200,"IMENA,IKIVI,ISEVU,IFURU,IRUGU,IDUBU",6,2022-04-06 13:11:37+00:00,2022-04-06,2022-04-06 13:11:37,2022-04-06 19:11:22+00:00,2022-04-06,19:11:22,guesses_sep0,IMENA
4,Burundi,"['UBURO', 'IZUBA', 'IDUBU']",5374,34308.0,IDUBU,11436,Africa/Bujumbura,True,2022-04-06 18:00:00 +0200,2022-04-06 21:10:37 +0200,"UBURO,IZUBA,IDUBU",3,2022-04-06 16:00:00+00:00,2022-04-06,2022-04-06 16:00:00,2022-04-06 19:10:37+00:00,2022-04-06,19:10:37,guesses_sep0,UBURO


In [10]:
df_reproduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24257 entries, 0 to 32259
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   country               24257 non-null  object             
 1   guesses               24257 non-null  object             
 2   id                    24257 non-null  int64              
 3   score                 24257 non-null  float64            
 4   solution              24257 non-null  object             
 5   time_taken            24257 non-null  int64              
 6   timezone              24257 non-null  object             
 7   won                   24257 non-null  bool               
 8   start_time            24257 non-null  object             
 9   end_time              24257 non-null  object             
 10  list_guesses          24257 non-null  object             
 11  count_guesses         24257 non-null  object             
 12  conv

In [11]:
path = os.getenv('path')
df_reproduced.to_csv(path + "Data/guesses-preprocessed.csv", index=False, header=True)

In [12]:
q6 = """ SELECT guesses_extracted
FROM df_reproduced
WHERE id = 5193
"""
print(ps.sqldf(q6, locals()))


  guesses_extracted
0             IRIBA
1             IMANA
2             IJURU
3             IDUBU


In [13]:
# popular words guessed 
q7 = """ SELECT guesses_extracted AS guess, COUNT(guesses_extracted) AS guess_freq
FROM df_reproduced

GROUP BY guesses_extracted

ORDER BY COUNT(guesses_extracted) DESC

"""

print(ps.sqldf(q7, locals()))

      guess  guess_freq
0     IJURU         941
1     IFURO         749
2     SOKWE         718
3     SHIMA         673
4     KANDI         647
...     ...         ...
1271  ABAZI           1
1272  ABASA           1
1273  ABAJA           1
1274  ABAHO           1
1275  ABAHE           1

[1276 rows x 2 columns]


In [14]:
# popular words guessed by country
q8 = """ SELECT country, guesses_extracted AS guess, COUNT(guesses_extracted) AS guess_freq
FROM df_reproduced

GROUP BY country, guesses_extracted

ORDER BY COUNT(guesses_extracted) DESC

"""

print(ps.sqldf(q8, locals()))

       country  guess  guess_freq
0      Burundi  IJURU         484
1      Burundi  SOKWE         375
2      Burundi  IFURO         361
3      Burundi  SHIMA         328
4      Burundi  KANDI         304
...        ...    ...         ...
4882  Zimbabwe  EREKA           1
4883  Zimbabwe  KIZWA           1
4884  Zimbabwe  KUGWA           1
4885  Zimbabwe  UMUTI           1
4886  Zimbabwe  URUGO           1

[4887 rows x 3 columns]


In [15]:
# popular 1st word guess by country
q9= """ SELECT country, guesses_extracted AS first_guess, COUNT(guesses_extracted) AS guess_freq
FROM df_reproduced
WHERE guesses_nber = 'guesses_sep0'

GROUP BY country, guesses_extracted

ORDER BY COUNT(guesses_extracted) DESC

"""

print(ps.sqldf(q9, locals()))

            country first_guess  guess_freq
0           Burundi       UMUTI         145
1           Burundi       AMATA         142
2           Burundi       IMANA         127
3           Burundi       AMAZI         120
4           Burundi       IMBWA         102
...             ...         ...         ...
1549  United States       YAMBU           1
1550         Zambia       GUKWA           1
1551         Zambia       ICUYA           1
1552         Zambia       UBURO           1
1553       Zimbabwe       UMUTI           1

[1554 rows x 3 columns]


In [16]:
# number of words guessed
q10 = """ SELECT COUNT(DISTINCT guesses_extracted)
FROM df_reproduced
"""
print(ps.sqldf(q10, locals()))

   COUNT(DISTINCT guesses_extracted)
0                               1276


In [17]:
# number of words solution
q11 = """ SELECT DISTINCT solution
FROM df_reproduced
"""
print(ps.sqldf(q11, locals()))

   solution
0     IDUBU
1     INKWI
2     SOKWE
3     INSWA
4     HEPFO
5     RIMWE
6     KANDI
7     INDWI
8     SHIMA
9     IJURU
10    IFURO
11    HANZE


In [18]:
# solution per day
q12 = """ SELECT country, start_date, solution 
FROM df_reproduced
GROUP BY start_date, solution, country
ORDER BY start_date DESC
"""
print(ps.sqldf(q12, locals()))

           country                  start_date solution
0          Belgium  2022-04-06 00:00:00.000000    IDUBU
1          Burundi  2022-04-06 00:00:00.000000    IDUBU
2         Cameroon  2022-04-06 00:00:00.000000    IDUBU
3           Canada  2022-04-06 00:00:00.000000    IDUBU
4            China  2022-04-06 00:00:00.000000    IDUBU
..             ...                         ...      ...
374  United States  1970-01-01 00:00:00.000000    INSWA
375        Burundi  1970-01-01 00:00:00.000000    KANDI
376         Canada  1970-01-01 00:00:00.000000    KANDI
377        Denmark  1970-01-01 00:00:00.000000    KANDI
378         France  1970-01-01 00:00:00.000000    KANDI

[379 rows x 3 columns]


In [17]:
#df2 = ps.sqldf(q7, locals())
#df2.info()

In [19]:
# popular guess
Counter = Counter(df_reproduced['guesses_extracted'])
  

most_occur = Counter.most_common(4)

print(most_occur)

[('IJURU', 941), ('IFURO', 749), ('SOKWE', 718), ('SHIMA', 673)]
